### 1. ĐỌC DỮ LIỆU HOÀN CHỈNH Ở TASK4

In [ ]:
import pandas as pd
df = pd.read_csv("../task04_merge/order_full_merged.csv")
df

In [4]:
import pandas as pd

# Đọc file đã merge ở nhiệm vụ 4
df = pd.read_csv("../task04_merge/order_full_merged.csv", parse_dates=["order_date"])

# Thêm cột tháng
df["thang"] = df["order_date"].dt.to_period("M").astype(str)

### 2. Pivot doanh thu theo category × tháng

In [5]:
pivot_dt = pd.pivot_table(
    df,
    index="category",   # hàng: danh mục
    columns="thang",    # cột: các tháng
    values="doanh_thu", # ô: tổng doanh thu
    aggfunc="sum",      # hàm tổng hợp: cộng doanh thu theo ô (category, tháng)
    fill_value=0        # thay NaN bằng 0 để bảng gọn gàng
)

print("Pivot doanh thu (category × tháng):")
print(pivot_dt)


Pivot doanh thu (category × tháng):
thang      2024-01   2024-02   2024-03   2024-04
category                                        
Gia Dụng   7900000  10100000  20550000   4150000
Điện Tử   19350000  20700000  13850000  11350000


### 3. Pivot tỷ lệ trả hàng theo category × sản phẩm

In [6]:
# 1) Tổng số đơn theo cặp (category, product_id)
tong_don = (
    df.groupby(["category","product_id"])["order_id"]
      .nunique()  # đếm đơn duy nhất
      .reset_index(name="tong_don")
)

# 2) Số đơn trả theo cặp (category, product_id)
don_tra = (
    df.loc[df["refund_amount"].notna()]  # chỉ lấy đơn có thông tin hoàn tiền
      .groupby(["category","product_id"])["order_id"]
      .nunique()
      .reset_index(name="don_tra")
)

# 3) Gộp và tính tỷ lệ trả hàng
ti_le = tong_don.merge(don_tra, on=["category","product_id"], how="left")
ti_le["don_tra"] = ti_le["don_tra"].fillna(0)           # nếu không có trả → 0
ti_le["ty_le_tra"] = ti_le["don_tra"] / ti_le["tong_don"]  # tỷ lệ = đơn trả / tổng đơn

# 4) Pivot tỷ lệ trả theo (category × product_id)
pivot_tra = pd.pivot_table(
    ti_le,
    index="category",    # hàng: danh mục
    columns="product_id",# cột: mã sản phẩm
    values="ty_le_tra",  # ô: tỷ lệ trả
    aggfunc="mean",      # mean ở đây chỉ hợp nhất khi có trùng; với mỗi ô duy nhất thì chính là tỷ lệ
    fill_value=0
)

print("\nPivot tỷ lệ trả hàng (category × product):")
print(pivot_tra)



Pivot tỷ lệ trả hàng (category × product):
product_id      PR01      PR02  PR03      PR04      PR05
category                                                
Gia Dụng    0.000000  0.000000  0.00  0.038462  0.107143
Điện Tử     0.090909  0.083333  0.05  0.000000  0.000000


### 4. Stack và unstack bảng pivot

In [7]:
# Stack: chuyển cột thành một cấp chỉ số (MultiIndex Series)
stacked = pivot_dt.stack()
print("\nStacked doanh thu (ví dụ 10 dòng đầu):")
print(stacked.head(10))

# Unstack: xoay lại về dạng DataFrame (ngược với stack)
unstacked = stacked.unstack()
print("\nUnstacked về lại DataFrame:")
print(unstacked.head())


Stacked doanh thu (ví dụ 10 dòng đầu):
category  thang  
Gia Dụng  2024-01     7900000
          2024-02    10100000
          2024-03    20550000
          2024-04     4150000
Điện Tử   2024-01    19350000
          2024-02    20700000
          2024-03    13850000
          2024-04    11350000
dtype: int64

Unstacked về lại DataFrame:
thang      2024-01   2024-02   2024-03   2024-04
category                                        
Gia Dụng   7900000  10100000  20550000   4150000
Điện Tử   19350000  20700000  13850000  11350000


### 5. Nhận xét danh mục có hiệu suất thấp nhất

In [8]:
# 1) Tổng doanh thu theo danh mục
dt_dm = df.groupby("category")["doanh_thu"].sum().reset_index()

# 2) Tỷ lệ trả hàng theo danh mục
tong_don_dm = df.groupby("category")["order_id"].nunique().reset_index(name="tong_don")
don_tra_dm  = (
    df.loc[df["refund_amount"].notna()]
      .groupby("category")["order_id"]
      .nunique()
      .reset_index(name="don_tra")
)
ti_le_dm = tong_don_dm.merge(don_tra_dm, on="category", how="left").fillna({"don_tra":0})
ti_le_dm["ty_le_tra"] = ti_le_dm["don_tra"] / ti_le_dm["tong_don"]

# 3) Gộp doanh thu & tỷ lệ trả để đánh giá
hieu_suat_dm = dt_dm.merge(ti_le_dm[["category","ty_le_tra"]], on="category")

# 4) Sắp xếp: doanh thu tăng (thấp lên trước), tỷ lệ trả giảm (cao lên trước)
kq = hieu_suat_dm.sort_values(["doanh_thu","ty_le_tra"], ascending=[True, False])

print("\nDoanh thu & tỷ lệ trả theo danh mục (xếp để phát hiện hiệu suất thấp):")
print(kq)


Doanh thu & tỷ lệ trả theo danh mục (xếp để phát hiện hiệu suất thấp):
   category  doanh_thu  ty_le_tra
0  Gia Dụng   42700000   0.074074
1   Điện Tử   65250000   0.075758
